In [9]:
import numpy as np
import matplotlib.pyplot as plt
import shutil
import subprocess
import math
# import pandas as pd
# import scipy as sp
# from scipy.io import loadmat
# import matlab.engine as eng


%------INPUTS-----%
% Code requires a 3D matrix of interpolated topography data.The data contains x-cordinates, y-cordinates, and z (elevation) values from a simulation.

%------OUTPUTS----%
% 1D arrays of sigmas_ss (standard deviation of the ratio of sedimentation to average aggardation rates over a given radial transect with in a basin.
% 1D arrays of variability in sigma_ss
% Exponent of a power law fit to the decay in sigma_ss as function of measurement time. The power law fit approach is motivated from Straub etal 2009

In [ ]:
dir = r'/---/---/---/---/---/----/'
deptick = loadmat('deptick1.mat')  ##Load 3D array that is (x,y,t)

In [10]:
def Syn_strata(z):      # This function creates synthetic stratigrapghy from a xsection of elevation data
    S_strat = z
    nx,nsteps = S_strat.shape
    for i in range(nx):
        for j in range(nsteps):
            S_strat[i,j]= np.min(S_strat[i,j:nsteps])                    
            return S_strat

def fsigma_ss(strata,drift_rate):    #Function Calculates sigma ss for range of measurment windows provided by data set, which is oriented with space x time
    nt = S_strat.shape[1]
    sss = []                 #list of sigma ss values
    sig_sss = []             #list standard deviation of sss
    for i in range(nt-1):    #loop with changing time window size
        sigU =[]             #list of individual sigma_ss measurements for a given window size
        for j in range(nt):
            line1 = S_strat[:,j]         #topography at time t=j
            if i+j > nt:
                break
            line2 = S_strat[:,i+j]                  #topography at time t = j+i
            diff = np.subtract(line2,line1)         #difference in line 2 and line 1 (i.e. sedimentation)
            drift = i*drift_rate                    #mean drift for this time window
            U = diff/drift                          #ratio of sedimentation/subsidence
            sigU.append(np.std(U))
        sttspan = np.nanmean(sigU)
        sig_sttspan = np.nanstd(sigU)
        sss.append(sttspan)
        sig_sss.append(sig_sttspan)
    return np.array(sss),np.array(sig_sss)         #Conver list to arrat for mathematical operations
        
nx = deptick.shape[0]
ny = deptick.shape[1]
nt = deptick.shape[2]
dx = 100                                           #grid node spacing in x direction (m)
dy = 100                                           #grid node spacing in y direction (m)
sssallradial = np.zeros((nt-1,nx))
ssserrorallradial = np.zeros((nt-1,nx))
xcenter = 1
ycenter = 188
num = 2*math.pi

for i in range(nx):
    th = np.arange(0,num,1/i)
    xunit = i*math.cos(th[i])+xcenter            #get x-locations on a radial transect
    yunit = i*math.sin(th[i])+ycenter            #get y-locations on a radial transect
    xunit = round(xunit)
    yunit = round(yunit)
    xunit2 = []
    yunit2 = []
    dx = np.max(xunit)
    for j in range(dx):
        xc = xunit
        yc = yunit
        if xc >= 1:
            if yc >= 1:
                if xc <= nx:
                    if yc <= ny:
                        xunit2.append(xc)
                        yunit2.append(yc)
    xunit = xunit2
    yunit = yunit2                     
    xs = []
    for k in range(dx):
        xs_shot = deptick[(yunit[k],xunit[k]),:]
        np.array(xs_shot)
        xs_shot = np.reshape(xs_shot,(2,3,1))  
        xs.append(xs_shot)
        
    z = xs
    strata = Syn_strata(z)                 #call function that corrects topography data for errosion
    dxx = strata.shape[0]
    for jj in range(dxx):                  #Loop to find western most location where deposition starts on xsection
        d = strata[jj,nt]-strata[jj,0]
        if d > 0.5:
            start = jj
            break

    for jj in range(dxx-1,-1,-1):          #Loop to find easter most location where deposition starts on xsection
        d = strata[jj,nt]-strata[jj,1]
        if d > 0.5:
            end = jj
            break

    if jj == 1:                           #If no deposition anywhere in xsection, note this through nan's
        sssallradial[:,i]= np.nan
        ssserrorallradial[:,i]= np.nan

    if jj > 1:                            #If deposition in xsection, calculate sigma ss decay
        Syn_strat= strata[start:end,:]    #clip xsection transect to limits of deposit
        zstart = strata[:,1]
        zend = strata[:,nt]
        temp = np.subtract(zend,zstart)
        drfit_rate = np.mean(temp)/nt        #calculate mean deposition rate for reach where deposition occurs
        sss,sig_sss = fsigma_ss_strat(strata,drift_rate)     #Run sigma ss function
        sssallradial[:,i]= sss
        ssserrorallradial[:,i]=sig_sss


NameError: name 'deptick' is not defined

In [11]:
##---Automate significant change detection across the mean for a given cross-section of stratigraphy---##
eng = matlab.engine.start_matlab()
nx = sig_sss.shape[0]
nt = sig_sss.shape[1]
dt = 1
t = range(dt,dt*nt)
Kappa = []
IND = []
RES =[]
for i in range(1,nx):
    sig_sss = sssallradial[:,i]
    ipt, residuals = eng.findchangepts(sig_ss, MaxNumChanges=1) 
    ind = ipt[0]
    IND.ppend(ind)
    tcrop = t[0:ind]
    tcrop.conj()
    ssscrop = sig_sss[0:ind]
    p = np.polyfit(np.log(tcrop),np.log(ssscrop),1)
    kappa = p[0]*-1
    Kappa.append(kappa)
    RES.append(residuals)   


NameError: name 'matlab' is not defined